In [65]:
#Import necessary libraries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MaxAbsScaler
from utils import isolation_forest_algorithm
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import os

In [66]:
#path of folders and datasets
sheet_names = []
xlsx_dolder = None
result_folder = None
cleaning_condition = None

In [67]:
#parameters
test_sheet = None

In [68]:
#A function that makes csv datasets from multi-sheet excels
def make_dataset(input_folder, output_folder, sheet_names):
    contents = os.listdir(input_folder)
    root = input_folder
    total_data = {}
    for sheet in sheet_names:
        total_data[sheet] = pd.DataFrame()
    for content in contents:
        adr = root + "/" + content
        if(os.path.isfile(adr)):
            file_data = pd.read_excel(adr, sheet_name = sheet_names)
            for sheet in sheet_names:
                total_data[sheet] = pd.concat([total_data[sheet], file_data[sheet]])
    for sheet in sheet_names:
        total_data[sheet] = total_data[sheet].reset_index(drop = True)
        total_data[sheet].to_csv(output_folder + "/" + sheet + ".csv", index = False)
    return total_data

In [69]:
#A function that loads all sheet datasets from csv files and puts them in a dict
def load_dataset(data_folder, sheet_names):
    data = {}
    for sheet in sheet_names:
        path = data_folder + "/" + sheet + ".csv"
        #with shuffling
        data[sheet] = pd.read_csv(path, low_memory=False).sample(frac = 1).reset_index(drop = True)
        #without shuffling
        #data[sheet] = pd.read_csv(path, low_memory=False)
    return data

In [70]:
#data = make_dataset(xlsx_folder, result_folder, sheet_names)

In [71]:
all_data = load_dataset("C:/Users/hamra/PycharmProjects/isolation_forest/data/result", sheet_names)

In [ ]:
print(len(all_data))

In [73]:
#A function that drops rows of the dataset which have null values in specific columns
def cleaning(dataframe):
    data = copy.deepcopy(dataframe)
    all_cols = data.columns
    cols = []
    for i in range(len(data.columns)):
        if(isinstance(data.iloc[0, i], str) or data.iloc[0, i] is np.NaN):
            pass
        else:
            cols.append(data.columns[i])
    data = data.dropna(subset = cols)
    data = data[cleaning_condition]
    print(cols)
    data.reset_index(drop = True, inplace = True)
    return data

In [74]:
def normalize(dataframe):
    return pd.DataFrame(data = MaxAbsScaler().fit_transform(dataframe))

In [ ]:
row_data = {}
data = {}
for sheet in sheet_names:
    row_data[sheet] = cleaning(all_data[sheet])
    data[sheet] = copy.deepcopy(row_data[sheet])

In [ ]:
all_data[test_sheet]

In [ ]:
row_data[test_sheet]

In [ ]:
#Replace string and null values in dataframes with 1 then normalizing them
for sheet in sheet_names:
    all_values = np.array([])
    for i in range(len(data[sheet].columns)):
        if(isinstance(data[sheet].iloc[0, i], str) or data[sheet].iloc[0, i] is np.NaN):
            col = data[sheet].columns[i]
            data[sheet].loc[:, col] = 1
            col_values = row_data[sheet][col].unique()
            all_values = np.concatenate((all_values, col_values))
    all_values = list(all_values)
    targets = list(np.full(len(all_values), 1))
    map = {all_values[i]: targets[i] for i in range(len(targets))}
    for item in list(map.keys()):
        pass
    data[sheet] = normalize(data[sheet])
data[test_sheet]

In [79]:
#in this block we implement last block with ordinal encoder
#for sheet in sheet_names:
#    cols = []
#    for i in range(len(data[sheet].columns)):
#        if(isinstance(data[sheet].iloc[0, i], str) or data[sheet].iloc[0, i] is np.NaN):
#            cols.append(data[sheet].columns[i])
#    data[sheet][cols] = OrdinalEncoder(encoded_missing_value = -0.5).fit_transform(data[sheet][cols])
#data[test_sheet]

In [ ]:
#Run the isolation forest algorithm on all sheets and show some results of the test sheet
#model = isolation_forest_algorithm.isolation_forest_algorithm(data[test_sheet], 0.01, flag = True)
#model.detection()
results = {}
for sheet in sheet_names:
    if(sheet == test_sheet):
        model = isolation_forest_algorithm.isolation_forest_algorithm(row_data[sheet], data[sheet], 0.01, flag = True)
    else:
        model = isolation_forest_algorithm.isolation_forest_algorithm(row_data[sheet], data[sheet], 0.01)
    results[sheet] = model.detection()
print(results)